In [21]:
import numpy as np
import open3d as o3d
import sys
import os

sys.path.append('..')
import open3d_tutorial as o3dtut
o3dtut.interactive = not 'CI' in os.environ

In [22]:
demo_data= o3d.data.DemoCropPointCloud()

In [23]:
pcd = o3d.io.read_point_cloud(demo_data.point_cloud_path)
pcd

PointCloud with 196133 points.

## Raw 3D model

In [24]:
o3d.visualization.draw_geometries([pcd])

## Pre-processing the 3D model
    1. We need to down sample the point
    2. We need to estimate the normal vectors of the points

In [26]:
## Down Sampling the points
down_pcd  = pcd.voxel_down_sample(voxel_size=0.05)
down_pcd

PointCloud with 4718 points.

    Visualizing the down sampling of the points

In [27]:
o3d.visualization.draw_geometries([down_pcd])

# Normal estimation
    We need normal estimation to get the vectors for each plane or surface 
    by doing so we can identify different objects from each other

In [28]:
down_pcd.estimate_normals(
    search_param= o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn=30)
)

In [33]:
o3d.visualization.draw_geometries([down_pcd],point_show_normal=True)

# Separation
    we will separate the objects from the give scene

In [35]:
vol = o3d.visualization.read_selection_polygon_volume(demo_data.cropped_json_path)

In [36]:
chair = vol.crop_point_cloud(down_pcd)


In [40]:
o3d.visualization.draw_geometries([chair])

    Separeting object and scene

In [38]:
dist  = down_pcd.compute_point_cloud_distance(chair)
dist = np.asarray(dist)
ind = np.where(dist>0.02)[0]

In [39]:
separated_scene = down_pcd.select_by_index(ind)
o3d.visualization.draw_geometries([separated_scene])

In [44]:
aabb = chair.get_axis_aligned_bounding_box()
aabb.color = (0,1,0)
obb = chair.get_oriented_bounding_box()
obb.color = (1,0,0)
o3d.visualization.draw_geometries([chair,aabb,obb])